# Dataset subsets to be integrated in the hierarchy - treeArches

### Load libraries and dataset

In [1]:
import os
os.chdir('../')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
import scanpy as sc
import numpy as np
import gdown
import copy as cp
import pandas as pd
import anndata as ad
import sys
from scipy.sparse import hstack, csc_matrix

In [3]:
#Import initial dataset (Sabatini mouse DVC)
adata = sc.read('allmice_adata_processed_2500.h5ad')
adata

AnnData object with n_obs × n_vars = 99740 × 2500
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ratio_nCount_nFeat', 'doubletfinder_class', 'seq_sample', 'treatment', 'single_r_celltypes', 'chen_celltypes', 'tasic_celltypes', 'romanov_celltypes', 'camp1_celltypes_full', 'identity_layer1', 'identity_layer2', 'identity_layer3', 'study', 'cellident_study', '_scvi_batch', '_scvi_labels'
    var: 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'mvp.variable', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'log1p'
    layers: 'counts', 'data'

We have generated a function to find the genes missing in the datasets compared to our murine dataset, and another function to append these with counts 0. These functions are applied to different datasets in this notebook.

### Ludwig mouse DVC database

In [4]:
adata_ludwig = sc.read('GSE166648.h5ad')
adata_ludwig

AnnData object with n_obs × n_vars = 72128 × 27998
    obs: 'Unnamed: 0', 'nCount_RNA', 'nFeature_RNA', 'seq_sample', 'percent.mito', 'percent.ribo', 'cell.type', 'neuronal.subtype', 'study', 'cell_barcode', 'treatment', 'identity_layer3'
    layers: 'counts'

In [5]:
#First function

def find_missing_genes(full_adata, subset_adata):
    ''' (AnnData, AnnData) -> list
    Takes as input a large AnnData object and a smaller AnnData object and finds which genes are in the small one but not in the large one.
    '''
    missing_genes = []
    for gene in subset_adata.var_names:
        if not gene in full_adata.var_names: missing_genes.append(gene)
    return missing_genes

In [6]:
genes_to_add = find_missing_genes(adata_ludwig, adata)
genes_to_add

['Gm37587',
 '2610203C22Rik',
 'Ecrg4',
 'Cfap65',
 'Gm20528',
 'E330020D12Rik',
 'Cd244a',
 'Ifi206',
 'Ifi214',
 'Ifi213',
 'Ifi209',
 'Ifi207',
 'Ifi211',
 '4930527J03Rik',
 'Gm13657',
 'Gm13684',
 'Knl1',
 'Morrbid',
 'Dnmt3c',
 'Gm11476',
 'Sox2ot',
 'Gm36823',
 'Gm6209',
 'Gask1b',
 'Tent5c',
 'Gm26871',
 'Gm17494',
 'Gm31243',
 'Gm35066',
 'Gm11867',
 'Spaar',
 'Gm12610',
 'Foxd2os',
 'Armh1',
 'Srarp',
 '6330411D24Rik',
 '5830444B04Rik',
 'Gm30835',
 'Gm38562',
 'Cfap299',
 'Grk3',
 'Pvrig',
 'Gm43378',
 'Plut',
 'Gm20559',
 'Trbc1',
 'Gimap1os',
 'Gsdme',
 '9130019P16Rik',
 'A530053G22Rik',
 'Gm20560',
 'Gm38843',
 'Gm32591',
 'AY512915',
 'Tafa1',
 'Depp1',
 'AC150683.1',
 'Gm45669',
 'Gm44752',
 'Gm29683',
 'B130024G19Rik',
 'Gm42397',
 'Gm44689',
 'Mir9-3hg',
 'Gm44751',
 'E230029C05Rik',
 'Gm32647',
 'Gm34280',
 'Gm38405',
 'Gm20663',
 'Gm44866',
 'Gm32916',
 'Gm5737',
 'Gm45184',
 'C030029H02Rik',
 'C230079O03Rik',
 'Gm36849',
 'Gm6249',
 'Gm16201',
 'Sox1ot',
 '4931415C1

In [7]:
#Second function

def add_missing_genes(adata, genes_to_add):
    ''' (AnnData, list) -> AnnData
    Takes as input an AnnData object and a list of gene names. Creates a new AnnData object which contains all the data from the input
    AnnData object with the genes from the list all added with blank values (0, False, None, etc. - depends on the specifical piece of
    data).
    '''
    #A new X is made for the new AnnData object. Empty columns are added to the X of the original AnnData.
    new_sparse_matrix = csc_matrix((len(adata.obs), len(genes_to_add)), dtype='float64')
    X_for_ad2 = hstack([adata.X, new_sparse_matrix])
    
    #A dataframe is made which has a row for each of the genes to add.
    new_vars = pd.DataFrame()
    new_vars.index = genes_to_add
    #The columns from the original AnnData's var are created with default values of 0 for floats and False for bools.
    i=0
    for column in adata.var.columns:
        if adata.var[column].dtypes == 'float64': new_vars.insert(i, column, [0]*len(genes_to_add), True)
        elif adata.var[column].dtypes == 'bool': new_vars.insert(i, column, [False]*len(genes_to_add), True)
        i+=1
    #The dataframe made above is appended to the original AnnData's var to make a new var.
    var_for_ad2 = adata.var.append(new_vars)
    
    #A new set of layers is created based on the layers of the original AnnData. 
    #Empty columns are added to each layer (as done for the new X) and a dictionary of new layers is created.
    layers_for_ad2 = {}
    for layer in adata.layers:
        new_sparse_matrix = csc_matrix((len(adata.obs), len(genes_to_add)), dtype='float64')
        new_layer = hstack([adata.layers[layer], new_sparse_matrix])
        layers_for_ad2[layer] = new_layer
    
    #A new AnnData is made based on the old one with the new X and var made above accounting for the missing genes.
    adata2 = ad.AnnData(X=X_for_ad2, 
                            obs=adata.obs, 
                            var=var_for_ad2, 
                            obsm=adata.obsm, 
                            obsp=adata.obsp, 
                            varm=adata.varm, 
                            varp=adata.varp, 
                            layers=layers_for_ad2, 
                            uns=adata.uns)
    return adata2

In [8]:
adata_ludwig = add_missing_genes(adata_ludwig, genes_to_add)
adata_ludwig

AnnData object with n_obs × n_vars = 72128 × 28254
    obs: 'Unnamed: 0', 'nCount_RNA', 'nFeature_RNA', 'seq_sample', 'percent.mito', 'percent.ribo', 'cell.type', 'neuronal.subtype', 'study', 'cell_barcode', 'treatment', 'identity_layer3'
    layers: 'counts'

In [9]:
#Check the genes after appending missing genes
adata_ludwig.var_names

Index(['Xkr4', 'Gm1992', 'Gm37381', 'Rp1', 'Rp1.1', 'Sox17', 'Gm37323',
       'Mrpl15', 'Lypla1', 'Gm37988',
       ...
       'BE692007', 'Gm50276', 'C730002L08Rik', 'Gm9895', 'Gm50323', 'Itprip',
       'Dipk2b', 'C430049B03Rik', 'P2ry10b', '2810403D21Rik'],
      dtype='object', length=28254)

In [10]:
#Save new object.
adata_ludwig.write('DVC_datasets_2500/ludwig_database_2500.h5ad')

### Dowsett mouse NTS dataset

We know by trying to subset this dataset in the treeArches pipeline, that the missing genes are 'AC150683.1', 'Sept4', '1010001N08Rik' and 'Tmem173'.

In [11]:
adata_dowsett = sc.read('final_nts_merged.h5ad')
adata_dowsett

AnnData object with n_obs × n_vars = 15931 × 55401
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percmt', 'ratio_nCount_nFeat', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.1', 'seurat_clusters', 'treatment', 'pANN_0.25_0.005_256', 'doubletfinder_class', 'pANN_0.25_0.005_273', 'seq_sample', 'cell_barcode'
    var: 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'mvp.variable'
    layers: 'counts', 'data'

In [12]:
genes_to_add_dowsett = find_missing_genes(adata_dowsett, adata)
genes_to_add_dowsett

['AC150683.1', 'Sept4', 'Hist1h2bc', '1010001N08Rik', 'Tmem173']

We now compare the original and just created objects. The new object has 4 more variables (the missing genes) and the rest matches exactly the original object.

In [13]:
adata_dowsett = add_missing_genes(adata_dowsett, genes_to_add_dowsett)
adata_dowsett

AnnData object with n_obs × n_vars = 15931 × 55406
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percmt', 'ratio_nCount_nFeat', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.1', 'seurat_clusters', 'treatment', 'pANN_0.25_0.005_256', 'doubletfinder_class', 'pANN_0.25_0.005_273', 'seq_sample', 'cell_barcode'
    var: 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'mvp.variable'
    layers: 'counts', 'data'

In [14]:
adata_dowsett.var_names

Index(['4933401J01Rik', 'Gm26206', 'Xkr4', 'Gm18956', 'Gm37180', 'Gm37363',
       'Gm37686', 'Gm1992', 'Gm37329', 'Gm7341',
       ...
       'mt-Nd6', 'mt-Te', 'mt-Cytb', 'mt-Tt', 'mt-Tp', 'AC150683.1', 'Sept4',
       'Hist1h2bc', '1010001N08Rik', 'Tmem173'],
      dtype='object', length=55406)

In [15]:
#Save new object.
adata_dowsett.write('DVC_datasets_2500/dowsett_data_2500.h5ad')

Now the dataset can be subset by the 2500 most variable genes contained in the Hes et al. dataset

### Hes - rat DVC database

In [16]:
adata_rat_Hes = sc.read('labeled_rat3.h5ad')
adata_rat_Hes

AnnData object with n_obs × n_vars = 12167 × 25339
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ratio_nCount_nFeat', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.1', 'RNA_snn_res.1.5', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.4', 'seurat_clusters', 'pANN_0.25_0.12_974', 'doubletfinder_class', 'identity_layer1', 'identity_layer2', 'identity_layer3', 'cell_barcode'
    var: 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'mvp.variable'
    layers: 'counts'

In [17]:
adata_rat_Hes.var_names

Index(['Vom2r3', 'LOC100909608', 'Vom2r6', 'Vom2r5', 'Raet1c',
       'AABR07000121.1', 'AABR07000137.1', 'AABR07000145.1', 'Raet1d',
       'AABR07000156.1',
       ...
       'AC242953.1', 'Eif2s3y', 'Uty', 'Ddx3y.1', 'Usp9y', 'AC241722.1',
       'LOC103694564', 'AC241722.2', 'LOC103694568', 'AC242615.1'],
      dtype='object', length=25339)

In [18]:
missing_rat = find_missing_genes(adata_rat_Hes, adata)
missing_rat

['Gm37587',
 '2610203C22Rik',
 '1700034P13Rik',
 'A830018L16Rik',
 'Gm28376',
 'Gm15457',
 'Ecrg4',
 'Slc40a1',
 'Gm28322',
 'Stat4',
 'BC055402',
 'Aox3',
 'Gm973',
 '6030407O03Rik',
 'Cfap65',
 'Gm816',
 'Col4a3',
 'Gm10552',
 'A630001G21Rik',
 'Gm20528',
 'Glrp1',
 'Iqca',
 'Crocc2',
 'Gm7967',
 '9330185C12Rik',
 'Cfhr2',
 'Gm20631',
 'E330020D12Rik',
 'Gm29291',
 'Gm28286',
 'A830008E24Rik',
 '4930523C07Rik',
 'Gm37273',
 'Sh2d1b1',
 'Fcgr3',
 'B930036N10Rik',
 'Cd244a',
 'Ifi206',
 'Ifi214',
 'Ifi213',
 'Ifi209',
 'Ifi207',
 'Ifi204',
 'Mndal',
 'Ifi211',
 'Ifi203',
 'Ifi202b',
 'Gm37306',
 '4930527J03Rik',
 'Gm37267',
 'Lefty1',
 '2210411M09Rik',
 '1700047M11Rik',
 'C130074G19Rik',
 'Prox1os',
 'Gm38171',
 '8030442B05Rik',
 'Gm13376',
 'Pnpla7',
 'Gm10134',
 'Cfap77',
 'AI182371',
 'Gm13470',
 'Gm13481',
 'Gm13657',
 'Pde11a',
 'Gm13684',
 'Olfr1030',
 'Olfr1033',
 'Gm10800',
 'Gm13905',
 'Wt1os',
 'Pax6os1',
 'Gm13912',
 'Gm13974',
 'Gm28494',
 'Knl1',
 'Morrbid',
 'Gm29010',
 '

In [19]:
adata_rat_Hes = add_missing_genes(adata_rat_Hes, missing_rat)

In [20]:
adata_rat_Hes.var_names

Index(['Vom2r3', 'LOC100909608', 'Vom2r6', 'Vom2r5', 'Raet1c',
       'AABR07000121.1', 'AABR07000137.1', 'AABR07000145.1', 'Raet1d',
       'AABR07000156.1',
       ...
       'mt-Co1', 'mt-Co2', 'mt-Atp8', 'mt-Atp6', 'mt-Co3', 'mt-Nd3', 'mt-Nd4l',
       'mt-Nd4', 'mt-Nd5', 'mt-Cytb'],
      dtype='object', length=26081)

In [21]:
adata_rat_Hes

AnnData object with n_obs × n_vars = 12167 × 26081
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ratio_nCount_nFeat', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.1', 'RNA_snn_res.1.5', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.4', 'seurat_clusters', 'pANN_0.25_0.12_974', 'doubletfinder_class', 'identity_layer1', 'identity_layer2', 'identity_layer3', 'cell_barcode'
    var: 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'mvp.variable'
    layers: 'counts'

In [22]:
#Save new object.
adata_rat_Hes.write('DVC_datasets_2500/rat_Hes_database_2500.h5ad')